In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, month, year, avg, max, when, countDistinct, sum as _sum, rank
from pyspark.sql.window import Window

# Create SparkSession
spark = SparkSession.builder \
    .appName("ProductOrdersAnalytics") \
    .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse") \
    .enableHiveSupport() \
    .getOrCreate()

# Sample order data
order_data = [
    (101, "Alice", "Laptop", "Electronics", 2, 50000, "2023-01-05"),
    (102, "Bob", "Jeans", "Clothing", 1, 2000, "2023-01-10"),
    (103, "Charlie", "Chair", "Furniture", 3, 3000, "2023-02-12"),
    (104, "David", "Smartphone", "Electronics", 1, 30000, "2023-01-20"),
    (105, "Eve", "Bookshelf", "Furniture", 2, 8000, "2023-03-18"),
    (106, "Frank", "Shirt", "Clothing", 5, 1500, "2023-02-22"),
    (107, "Grace", "Tablet", "Electronics", 2, 25000, "2023-01-25"),
    (108, "Heidi", "Novel", "Books", 4, 500, "2023-03-03"),
    (109, "Ivan", "Textbook", "Books", 1, 1200, "2023-03-05"),
    (110, "Judy", "Sofa", "Furniture", 1, 20000, "2023-01-15"),
    (111, "Mallory", "Dress", "Clothing", 2, 2500, "2023-03-20"),
    (112, "Niaj", "Notebook", "Books", 3, 800, "2023-02-11"),
]

columns = ["OrderID", "CustomerName", "Product", "Category", "Quantity", "UnitPrice", "OrderDate"]
df = spark.createDataFrame(order_data, columns)

# Create views
df.createOrReplaceTempView("orders_local")
df.createOrReplaceGlobalTempView("orders_global")


In [2]:
# 1. Electronics orders with Quantity >= 2
spark.sql("""
    SELECT * FROM orders_local
    WHERE Category = 'Electronics' AND Quantity >= 2
""").show()

# 2. Add TotalAmount column (Quantity * UnitPrice)
spark.sql("""
    SELECT *, Quantity * UnitPrice AS TotalAmount
    FROM orders_local
""").show()

# 3. Total number of orders per Category
spark.sql("""
    SELECT Category, COUNT(*) AS OrderCount
    FROM orders_local
    GROUP BY Category
""").show()

# 4. Orders placed in January 2023
spark.sql("""
    SELECT * FROM orders_local
    WHERE OrderDate LIKE '2023-01-%'
""").show()

# 5. Average UnitPrice per Category
spark.sql("""
    SELECT Category, AVG(UnitPrice) AS AvgPrice
    FROM orders_local
    GROUP BY Category
""").show()

# 6. Order with the highest total amount
spark.sql("""
    SELECT *, Quantity * UnitPrice AS TotalAmount
    FROM orders_local
    ORDER BY TotalAmount DESC
    LIMIT 1
""").show()

# 7. Drop local view and try querying again
spark.catalog.dropTempView("orders_local")
try:
    spark.sql("SELECT * FROM orders_local").show()
except Exception as e:
    print("Error after dropping local view:", e)


+-------+------------+-------+-----------+--------+---------+----------+
|OrderID|CustomerName|Product|   Category|Quantity|UnitPrice| OrderDate|
+-------+------------+-------+-----------+--------+---------+----------+
|    101|       Alice| Laptop|Electronics|       2|    50000|2023-01-05|
|    107|       Grace| Tablet|Electronics|       2|    25000|2023-01-25|
+-------+------------+-------+-----------+--------+---------+----------+

+-------+------------+----------+-----------+--------+---------+----------+-----------+
|OrderID|CustomerName|   Product|   Category|Quantity|UnitPrice| OrderDate|TotalAmount|
+-------+------------+----------+-----------+--------+---------+----------+-----------+
|    101|       Alice|    Laptop|Electronics|       2|    50000|2023-01-05|     100000|
|    102|         Bob|     Jeans|   Clothing|       1|     2000|2023-01-10|       2000|
|    103|     Charlie|     Chair|  Furniture|       3|     3000|2023-02-12|       9000|
|    104|       David|Smartphone|

In [3]:
# 1. Furniture orders with TotalAmount > 10,000
spark.sql("""
    SELECT *, Quantity * UnitPrice AS TotalAmount
    FROM global_temp.orders_global
    WHERE Category = 'Furniture' AND (Quantity * UnitPrice) > 10000
""").show()

# 2. Add DiscountFlag column
spark.sql("""
    SELECT *,
        CASE WHEN Quantity > 3 THEN 'Yes' ELSE 'No' END AS DiscountFlag
    FROM global_temp.orders_global
""").show()

# 3. Customers who ordered more than 1 product type
spark.sql("""
    SELECT CustomerName
    FROM global_temp.orders_global
    GROUP BY CustomerName
    HAVING COUNT(DISTINCT Category) > 1
""").show()

# 4. Count number of orders per month
spark.sql("""
    SELECT
        CONCAT(YEAR(TO_DATE(OrderDate)), '-', LPAD(MONTH(TO_DATE(OrderDate)), 2, '0')) AS YearMonth,
        COUNT(*) AS OrderCount
    FROM global_temp.orders_global
    GROUP BY YearMonth
    ORDER BY YearMonth
""").show()

# 5. Rank all products by total quantity sold
ranked_df = df.groupBy("Product") \
    .agg(_sum("Quantity").alias("TotalQuantity")) \
    .withColumn("Rank", rank().over(Window.orderBy(col("TotalQuantity").desc())))

ranked_df.show()

# 6. Query global view using new SparkSession
new_spark = SparkSession.builder.appName("NewSession").getOrCreate()
new_spark.sql("SELECT * FROM global_temp.orders_global WHERE Category = 'Books'").show()


+-------+------------+---------+---------+--------+---------+----------+-----------+
|OrderID|CustomerName|  Product| Category|Quantity|UnitPrice| OrderDate|TotalAmount|
+-------+------------+---------+---------+--------+---------+----------+-----------+
|    105|         Eve|Bookshelf|Furniture|       2|     8000|2023-03-18|      16000|
|    110|        Judy|     Sofa|Furniture|       1|    20000|2023-01-15|      20000|
+-------+------------+---------+---------+--------+---------+----------+-----------+

+-------+------------+----------+-----------+--------+---------+----------+------------+
|OrderID|CustomerName|   Product|   Category|Quantity|UnitPrice| OrderDate|DiscountFlag|
+-------+------------+----------+-----------+--------+---------+----------+------------+
|    101|       Alice|    Laptop|Electronics|       2|    50000|2023-01-05|          No|
|    102|         Bob|     Jeans|   Clothing|       1|     2000|2023-01-10|          No|
|    103|     Charlie|     Chair|  Furniture

In [4]:
# 1. Save "Books" only as new global temp view
books_df = df.filter(col("Category") == "Books")
books_df.createOrReplaceGlobalTempView("books_orders")

# 2. Most purchased product per category
windowSpec = Window.partitionBy("Category").orderBy(col("TotalQty").desc())

df.groupBy("Category", "Product") \
  .agg(_sum("Quantity").alias("TotalQty")) \
  .withColumn("Rank", rank().over(windowSpec)) \
  .filter(col("Rank") == 1) \
  .select("Category", "Product", "TotalQty") \
  .show()

# 3. View that excludes all Clothing orders → "filtered_orders"
df.filter(col("Category") != "Clothing") \
  .createOrReplaceTempView("filtered_orders")

# To verify:
spark.sql("SELECT DISTINCT Category FROM filtered_orders").show()


+-----------+-------+--------+
|   Category|Product|TotalQty|
+-----------+-------+--------+
|      Books|  Novel|       4|
|   Clothing|  Shirt|       5|
|Electronics| Laptop|       2|
|Electronics| Tablet|       2|
|  Furniture|  Chair|       3|
+-----------+-------+--------+

+-----------+
|   Category|
+-----------+
|Electronics|
|  Furniture|
|      Books|
+-----------+

